# SWR

A flexible framework for safe withdrawal rate experiments.

Framework can generalize to
- Any generator of asset returns
- Any asset allocation strategy based on age, returns etc.
- Any utility function to evaluate suitability of strategy (e.g. total spending, certainty equivalent spending)
- Support a survival table 
- Any optimizer to find optimal parameters for a given withdrawal framework and market simulation


In [1]:
#import sys
#sys.path.append("../")
import pytest
import numpy as np
import pandas as pd

from SWRsimulation.SWRsimulationCE import SWRsimulationCE


In [2]:
# load from pickle
RETURN_FILE = '../data/histretSP'
def load_returns():
    return pd.read_pickle('%s.pickle' % RETURN_FILE)

download_df = load_returns()
return_df = download_df.iloc[:, [0, 3, 16]]
return_df.columns=['stocks', 'bonds', 'cpi']
return_df

,stocks,bonds,cpi
Year,,,
1928,0.438112,0.032196,-0.011561
1929,-0.082979,0.030179,0.005848
1930,-0.251236,0.005398,-0.063953
1931,-0.438375,-0.156808,-0.093168
1932,-0.086424,0.235896,-0.102740
...,...,...,...
2018,-0.042269,-0.027626,0.019102
2019,0.312117,0.153295,0.022851
2020,0.180232,0.104115,0.013620


In [3]:
# should adjust CPI to year-ending also but leave it for now
real_return_df = return_df.copy()
# real_return_df.loc[1948:, 'cpi'] = cpi_test['cpi_fred']
# adjust returns for inflation
real_return_df['stocks'] = (1 + real_return_df['stocks']) / (1 + real_return_df['cpi']) - 1
real_return_df['bonds'] = (1 + real_return_df['bonds']) / (1 + real_return_df['cpi']) - 1
real_return_df.drop('cpi', axis=1, inplace=True)
real_return_df

,stocks,bonds
Year,,
1928,0.454932,0.044268
1929,-0.088311,0.024189
1930,-0.200079,0.074090
1931,-0.380674,-0.070178
1932,0.018184,0.377411
...,...,...
2018,-0.060220,-0.045852
2019,0.282803,0.127529
2020,0.164373,0.089279


In [4]:
# zero returns, zero spending (just check shape)
RETURN = 0.0
# spending
FIXED = 0
VARIABLE = 0.0
NYEARS = 30

s = SWRsimulationCE({
    'simulation': {'returns_df': pd.DataFrame({'stocks': np.zeros(len(real_return_df)), 
                                               'bonds': np.zeros(len(real_return_df))}, 
                                              index=real_return_df.index),
                   'n_ret_years': NYEARS,
                  },
    'allocation': {'asset_weights': np.array([0.5, 0.5])}, 
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': 0,
                  },
    'evaluation': {'gamma': 0},
    'visualization': {}    
})
s.simulate()

print(s)

# just simulate 1st year in trials
z = s.latest_simulation[0]['trial']
assert len(z) == 30
assert(z.index[0]) == 1928, "start year == 1928"
assert(z.index[-1]) == 1957, "end year == 1957"

z

Simulation:
{'n_asset_years': 95,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':       stocks  bonds
Year               
1928     0.0    0.0
1929     0.0    0.0
1930     0.0    0.0
1931     0.0    0.0
1932     0.0    0.0
...      ...    ...
2018     0.0    0.0
2019     0.0    0.0
2020     0.0    0.0
2021     0.0    0.0
2022     0.0    0.0

[95 rows x 2 columns],
 'trials': <generator object SWRsimulationCE.historical_trials at 0x120a23140>}

Allocation:
{'asset_weights': array([0.5, 0.5])}

Withdrawal:
{'fixed': 0.0,
 'fixed_pct': 0,
 'floor': 0.0,
 'floor_pct': 0,
 'invest_expense': 0.0,
 'variable': 0.0,
 'variable_pct': 0.0}


,start_port,port_return,before_spend,spend,end_port,alloc_0,alloc_1
1928,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1929,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1930,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1931,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1932,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1933,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1934,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1935,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1936,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1937,100.0,0.0,100.0,0.0,100.0,0.5,0.5


In [5]:
# zero returns, spend 2% of starting portfolio per year, check ending value declines to 0.4
RETURN = 0.0
# spending
FIXED = 2.0
VARIABLE = 0.00
NYEARS = 30

returns_df = pd.DataFrame(index=range(1928, 2021), data={'stocks': RETURN, 'bonds': RETURN})


s = SWRsimulationCE({
    'simulation': {'returns_df': pd.DataFrame({'stocks': np.zeros(len(real_return_df)), 
                                               'bonds': np.zeros(len(real_return_df))}, 
                                              index=real_return_df.index),
                   'n_ret_years': NYEARS,
                  },
    'allocation': {'asset_weights': np.array([0.5, 0.5])}, 
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': 0,
                  },
    'evaluation': {'gamma': 0},
    'visualization': {}    
})

print(s)

z = s.simulate_trial(next(s.simulation['trials']))

assert(z['start_port'].iloc[0]) == 100, "start port value == 100"
assert(z['end_port'].iloc[-1]) == 40, "ending port value == 40"

z

Simulation:
{'n_asset_years': 95,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':       stocks  bonds
Year               
1928     0.0    0.0
1929     0.0    0.0
1930     0.0    0.0
1931     0.0    0.0
1932     0.0    0.0
...      ...    ...
2018     0.0    0.0
2019     0.0    0.0
2020     0.0    0.0
2021     0.0    0.0
2022     0.0    0.0

[95 rows x 2 columns],
 'trials': <generator object SWRsimulationCE.historical_trials at 0x120a23450>}

Allocation:
{'asset_weights': array([0.5, 0.5])}

Withdrawal:
{'fixed': 2.0,
 'fixed_pct': 2.0,
 'floor': 0.0,
 'floor_pct': 0,
 'invest_expense': 0.0,
 'variable': 0.0,
 'variable_pct': 0.0}


,start_port,port_return,before_spend,spend,end_port,alloc_0,alloc_1
1928,100.0,0.0,100.0,2.0,98.0,0.5,0.5
1929,98.0,0.0,98.0,2.0,96.0,0.5,0.5
1930,96.0,0.0,96.0,2.0,94.0,0.5,0.5
1931,94.0,0.0,94.0,2.0,92.0,0.5,0.5
1932,92.0,0.0,92.0,2.0,90.0,0.5,0.5
1933,90.0,0.0,90.0,2.0,88.0,0.5,0.5
1934,88.0,0.0,88.0,2.0,86.0,0.5,0.5
1935,86.0,0.0,86.0,2.0,84.0,0.5,0.5
1936,84.0,0.0,84.0,2.0,82.0,0.5,0.5
1937,82.0,0.0,82.0,2.0,80.0,0.5,0.5


In [6]:
# zero returns, spend 2% of current portfolio per year, check ending value declines to 0.98 ** 30
RETURN = 0.0
FIXED = 0
VARIABLE = 2.0
NYEARS = 30

s = SWRsimulationCE({
    'simulation': {'returns_df': pd.DataFrame({'stocks': np.zeros(len(real_return_df)), 
                                               'bonds': np.zeros(len(real_return_df))}, 
                                              index=real_return_df.index),
                   'n_ret_years': NYEARS,
                  },
    'allocation': {'asset_weights': np.array([0.5, 0.5])}, 
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': 0,
                  },
    'evaluation': {'gamma': 0},
    'visualization': {}    
})

print(s)

z = s.simulate_trial(next(s.simulation['trials']))

assert(z['start_port'].iloc[0]) == 100, "start port value == 100"
assert z['end_port'].iloc[-1] == pytest.approx(100 * ((1 - VARIABLE/100) ** NYEARS), 0.000001)
z

Simulation:
{'n_asset_years': 95,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':       stocks  bonds
Year               
1928     0.0    0.0
1929     0.0    0.0
1930     0.0    0.0
1931     0.0    0.0
1932     0.0    0.0
...      ...    ...
2018     0.0    0.0
2019     0.0    0.0
2020     0.0    0.0
2021     0.0    0.0
2022     0.0    0.0

[95 rows x 2 columns],
 'trials': <generator object SWRsimulationCE.historical_trials at 0x120a237d0>}

Allocation:
{'asset_weights': array([0.5, 0.5])}

Withdrawal:
{'fixed': 0.0,
 'fixed_pct': 0,
 'floor': 0.0,
 'floor_pct': 0,
 'invest_expense': 0.0,
 'variable': 0.02,
 'variable_pct': 2.0}


,start_port,port_return,before_spend,spend,end_port,alloc_0,alloc_1
1928,100.000000,0.0,100.000000,2.000000,98.000000,0.5,0.5
1929,98.000000,0.0,98.000000,1.960000,96.040000,0.5,0.5
1930,96.040000,0.0,96.040000,1.920800,94.119200,0.5,0.5
1931,94.119200,0.0,94.119200,1.882384,92.236816,0.5,0.5
1932,92.236816,0.0,92.236816,1.844736,90.392080,0.5,0.5
1933,90.392080,0.0,90.392080,1.807842,88.584238,0.5,0.5
1934,88.584238,0.0,88.584238,1.771685,86.812553,0.5,0.5
1935,86.812553,0.0,86.812553,1.736251,85.076302,0.5,0.5
1936,85.076302,0.0,85.076302,1.701526,83.374776,0.5,0.5
1937,83.374776,0.0,83.374776,1.667496,81.707281,0.5,0.5


In [7]:
# 4% real return, spend fixed 4% of starting, assert ending value unchanged
RETURN = 0.04
FIXED = 4 
FLOOR_PCT = 0.0
VARIABLE = 0.0
NYEARS = 30

returns_df = pd.DataFrame(index=real_return_df.index, data={'stocks': RETURN, 'bonds': RETURN})

s = SWRsimulationCE({
    'simulation': {'returns_df': returns_df,
                   'n_ret_years': NYEARS,
                  },
    'allocation': {},  # no args, default equal weight
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': FLOOR_PCT},
    'evaluation': {'gamma': 0},
})

print(s)

z = s.simulate_trial(next(s.simulation['trials']))

assert(z['start_port'].iloc[0]) == 100, "start port value == 100"
assert(z['end_port'].iloc[-1]) == 100, "end port value correct"
z


Simulation:
{'n_asset_years': 95,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':       stocks  bonds
Year               
1928    0.04   0.04
1929    0.04   0.04
1930    0.04   0.04
1931    0.04   0.04
1932    0.04   0.04
...      ...    ...
2018    0.04   0.04
2019    0.04   0.04
2020    0.04   0.04
2021    0.04   0.04
2022    0.04   0.04

[95 rows x 2 columns],
 'trials': <generator object SWRsimulationCE.historical_trials at 0x120a23990>}

Allocation:
{'asset_weights': array([0.5, 0.5])}

Withdrawal:
{'fixed': 4.0,
 'fixed_pct': 4,
 'floor': 0.0,
 'floor_pct': 0.0,
 'invest_expense': 0.0,
 'variable': 0.0,
 'variable_pct': 0.0}


,start_port,port_return,before_spend,spend,end_port,alloc_0,alloc_1
1928,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1929,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1930,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1931,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1932,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1933,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1934,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1935,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1936,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1937,100.0,0.04,104.0,4.0,100.0,0.5,0.5


In [8]:
# return 0.02% variable spending 0.02/1.02, check final value unchanged
RETURN = 0.02
FIXED = 0.0
FLOOR = 0.0
VARIABLE = 0.02/1.02*100
NYEARS = 30

returns_df = pd.DataFrame(index=real_return_df.index, data={'stocks': RETURN, 'bonds': RETURN})

s = SWRsimulationCE({
    'simulation': {'returns_df': returns_df,
                   'n_ret_years': NYEARS,
                  },
    'allocation': {},  # no args, default equal weight
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': FLOOR},
    'evaluation': {'gamma': 0},
})

print(s)

z = s.simulate_trial(next(s.simulation['trials']))

assert (z['start_port'].iloc[0]) == 100, "start port value == 100"
assert (z['end_port'].iloc[-1]) == 100, "end port value correct"
z


Simulation:
{'n_asset_years': 95,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':       stocks  bonds
Year               
1928    0.02   0.02
1929    0.02   0.02
1930    0.02   0.02
1931    0.02   0.02
1932    0.02   0.02
...      ...    ...
2018    0.02   0.02
2019    0.02   0.02
2020    0.02   0.02
2021    0.02   0.02
2022    0.02   0.02

[95 rows x 2 columns],
 'trials': <generator object SWRsimulationCE.historical_trials at 0x120a23c30>}

Allocation:
{'asset_weights': array([0.5, 0.5])}

Withdrawal:
{'fixed': 0.0,
 'fixed_pct': 0.0,
 'floor': 0.0,
 'floor_pct': 0.0,
 'invest_expense': 0.0,
 'variable': 0.0196078431372549,
 'variable_pct': 1.9607843137254901}


,start_port,port_return,before_spend,spend,end_port,alloc_0,alloc_1
1928,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1929,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1930,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1931,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1932,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1933,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1934,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1935,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1936,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1937,100.0,0.02,102.0,2.0,100.0,0.5,0.5


In [9]:
# check values per appendix of Bengen paper https://www.retailinvestor.org/pdf/Bengen1.pdf
# nominal return 10% for stocks, 5% for bonds
# inflation 3%
# fixed spending of 4% of orig port
STOCK_RETURN = (1.1 / 1.03) - 1
BOND_RETURN = (1.05 / 1.03) - 1
VARIABLE = 0.0
FIXED = 4.0
FLOOR = 0.0
NYEARS = 30

returns_df = pd.DataFrame(index=real_return_df.index, data={'stocks': STOCK_RETURN, 'bonds': BOND_RETURN})

s = SWRsimulationCE({
    'simulation': {'returns_df': returns_df,
                   'n_ret_years': NYEARS,
                  },
    'allocation': {},  # no args, default equal weight
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': FLOOR},
    'evaluation': {'gamma': 0},
})

print(s)

z = s.simulate_trial(next(s.simulation['trials']))

# match figures in appendix
# example uses nominal vals with 3% inflation, we use real vals
assert z.iloc[0]['before_spend'] * 1.03 == pytest.approx(107.5, 0.000001)
assert z.iloc[0]['spend'] * 1.03 == 4.12, "spend does not match Bengen"
assert z.iloc[0]['end_port'] * 1.03 == pytest.approx(103.38, 0.000001), "ending port does not match Bengen"

z

Simulation:
{'n_asset_years': 95,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':         stocks     bonds
Year                    
1928  0.067961  0.019417
1929  0.067961  0.019417
1930  0.067961  0.019417
1931  0.067961  0.019417
1932  0.067961  0.019417
...        ...       ...
2018  0.067961  0.019417
2019  0.067961  0.019417
2020  0.067961  0.019417
2021  0.067961  0.019417
2022  0.067961  0.019417

[95 rows x 2 columns],
 'trials': <generator object SWRsimulationCE.historical_trials at 0x120a23df0>}

Allocation:
{'asset_weights': array([0.5, 0.5])}

Withdrawal:
{'fixed': 4.0,
 'fixed_pct': 4.0,
 'floor': 0.0,
 'floor_pct': 0.0,
 'invest_expense': 0.0,
 'variable': 0.0,
 'variable_pct': 0.0}


,start_port,port_return,before_spend,spend,end_port,alloc_0,alloc_1
1928,100.000000,0.043689,104.368932,4.0,100.368932,0.5,0.5
1929,100.368932,0.043689,104.753982,4.0,100.753982,0.5,0.5
1930,100.753982,0.043689,105.155855,4.0,101.155855,0.5,0.5
1931,101.155855,0.043689,105.575286,4.0,101.575286,0.5,0.5
1932,101.575286,0.043689,106.013041,4.0,102.013041,0.5,0.5
1933,102.013041,0.043689,106.469922,4.0,102.469922,0.5,0.5
1934,102.469922,0.043689,106.946763,4.0,102.946763,0.5,0.5
1935,102.946763,0.043689,107.444437,4.0,103.444437,0.5,0.5
1936,103.444437,0.043689,107.963854,4.0,103.963854,0.5,0.5
1937,103.963854,0.043689,108.505964,4.0,104.505964,0.5,0.5


In [18]:
VARIABLE = 0.0
FIXED = 4.0
NYEARS = 30
FLOOR = 0.0
INVEST_EXPENSE = 0.0

s = SWRsimulationCE({
    'simulation': {'returns_df': real_return_df,
                   'n_ret_years': NYEARS,
                  },
#    'allocation': {},  # no args, default equal weight
    'allocation': {'asset_weights': np.array([0.60, 0.40])}, 
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': FLOOR,
                   'invest_expense': INVEST_EXPENSE},
    'evaluation': {'gamma': 0},
})

print(s)

z = s.simulate()

Simulation:
{'n_asset_years': 95,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':         stocks     bonds
Year                    
1928  0.454932  0.044268
1929 -0.088311  0.024189
1930 -0.200079  0.074090
1931 -0.380674 -0.070178
1932  0.018184  0.377411
...        ...       ...
2018 -0.060220 -0.045852
2019  0.282803  0.127529
2020  0.164373  0.089279
2021  0.199522 -0.057578
2022 -0.229552 -0.196470

[95 rows x 2 columns],
 'trials': <generator object SWRsimulationCE.historical_trials at 0x1302d9bd0>}

Allocation:
{'asset_weights': array([0.6, 0.4])}

Withdrawal:
{'fixed': 4.0,
 'fixed_pct': 4.0,
 'floor': 0.0,
 'floor_pct': 0.0,
 'invest_expense': 0.0,
 'variable': 0.0,
 'variable_pct': 0.0}


In [19]:
z

[{'trial':       start_port  port_return  before_spend  spend    end_port  alloc_0  \
  1928  100.000000     0.290666    129.066614    4.0  125.066614      0.6   
  1929  125.066614    -0.043311    119.649857    4.0  115.649857      0.6   
  1930  115.649857    -0.090411    105.193802    4.0  101.193802      0.6   
  1931  101.193802    -0.256476     75.240023    4.0   71.240023      0.6   
  1932   71.240023     0.161875     82.771995    4.0   78.771995      0.6   
  1933   78.771995     0.341520    105.674230    4.0  101.674230      0.6   
  1934  101.674230     0.052192    106.980815    4.0  102.980815      0.6   
  1935  102.980815     0.295016    133.361835    4.0  129.361835      0.6   
  1936  129.361835     0.219521    157.759532    4.0  153.759532      0.6   
  1937  153.759532    -0.251083    115.153165    4.0  111.153165      0.6   
  1938  111.153165     0.247286    138.639823    4.0  134.639823      0.6   
  1939  134.639823     0.025347    138.052560    4.0  134.052560   

In [20]:
import plotly.express as px
from plotly import graph_objects as go

In [21]:
years_survived = pd.DataFrame(data={'nyears': [30 - len(np.where(y['trial']['spend']==0.0)[0]) for y in z]},
                              index=range(1928,1994)).reset_index()

px.bar(years_survived, x="index", y="nyears", color="nyears",
              hover_name="index", color_continuous_scale="spectral")


In [22]:
portval_df = pd.DataFrame(data=np.hstack([(np.ones(66).reshape(66, 1) * 100), np.array([y['trial']['end_port'].values for y in z])])) \
    .transpose()
portval_df.columns=range(1928,1994)
portval_df['mean'] = portval_df.mean(axis=1)
portval_df

col_list = list(portval_df.columns)
portval_df.reset_index(inplace=True)
portval_melt = pd.melt(portval_df, id_vars=['index'], value_vars=col_list)
portval_melt.columns=['ret_year', 'start_year', 'portval']
portval_melt

,ret_year,start_year,portval
0,0,1928,100.000000
1,1,1928,125.066614
2,2,1928,115.649857
3,3,1928,101.193802
4,4,1928,71.240023
...,...,...,...
2072,26,mean,223.139321
2073,27,mean,229.123175
2074,28,mean,236.612235
2075,29,mean,246.724630


In [23]:
portval_df

,index,1928,1929,1930,1931,1932,1933,1934,1935,1936,...,1985,1986,1987,1988,1989,1990,1991,1992,1993,mean
0,0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,1,125.066614,91.668903,86.958869,70.352403,112.187490,130.152030,101.219204,125.501631,117.952144,...,119.591320,114.391137,95.970490,107.060141,115.640905,90.826973,117.557440,102.282263,105.542170,102.398840
2,2,115.649857,79.380997,60.656008,77.740691,146.501795,132.944929,127.080519,149.051930,84.336395,...,137.585524,110.357380,102.584961,124.087721,105.658769,106.406943,120.942708,108.042210,99.067923,104.429164
3,3,101.193802,55.021679,66.474694,100.290715,150.148022,168.165852,150.977419,107.627561,101.191632,...,133.544922,118.563062,118.733576,113.668630,124.436096,109.091707,128.483266,101.509354,121.966814,106.630710
4,4,71.240023,59.928307,85.177151,101.525092,190.444137,201.081862,109.069593,130.242385,99.756553,...,144.315179,137.849920,108.591456,134.172677,128.253498,115.501423,121.471206,125.071142,132.371985,109.442923
5,5,78.771995,76.395041,85.622720,127.476650,228.250709,146.593675,132.041011,129.543660,92.132587,...,168.659986,126.718906,128.000994,138.601757,136.491665,108.793699,150.453030,135.842957,158.169618,113.236130
6,6,101.674230,76.382253,106.882818,151.460508,166.940893,178.844285,131.387876,120.837585,75.067230,...,155.935159,150.036259,132.042353,147.827372,129.291862,134.333330,164.222631,162.421923,183.074305,117.335416
7,7,102.980815,94.916264,126.345889,109.431386,204.223092,179.377480,122.614804,99.701561,74.195683,...,185.550787,155.461931,140.642058,140.361824,160.397148,146.199077,197.190012,188.103698,197.231011,121.922097
8,8,129.361835,111.752419,90.622615,132.492271,205.399568,168.861037,101.226752,99.856659,81.218589,...,193.207575,166.296372,133.344957,174.472822,175.341222,175.109260,229.225477,202.759202,183.551290,126.654656
9,9,153.759532,79.693314,109.032349,131.850575,193.937791,140.914790,101.445413,110.691895,86.544065,...,207.643770,158.397852,165.550738,191.079336,210.811456,203.109581,247.959304,188.808168,169.548926,130.364554


In [24]:
fig = go.Figure()
for year in range(1928,1994):
    
    fig.add_trace(go.Scatter(x=portval_df['index'], 
                             y=portval_df[year],
                             mode='lines',
                             name=str(year),
                             line={'width': 1},
                            ),
                 )

fig.add_trace(go.Scatter(x=portval_df['index'], 
                         y=portval_df['mean'],
                         mode='lines',
                         name='Mean',
                         line={'width': 3, 'color': 'black'},
                        ),
             )
    
fig.update_layout(showlegend=False,
                  plot_bgcolor="white",
                  title="Retirement Outcomes, 1928-1994",
                  xaxis=dict(title="Retirement Year", linecolor='black', mirror=True, ticks='inside',),
                  yaxis=dict(title="Portfolio Value", linecolor='black', mirror=True, ticks='inside'),
                 )
fig.show()


In [ ]:
fig = px.line(portval_melt, x="ret_year", y="portval", color="start_year",
              hover_name="start_year")
fig.show()

In [ ]:
VARIABLE = 0.0
FIXED = 4.0
FLOOR = 0.0
NYEARS = 30
NTRIALS = 10000

s = SWRsimulationCE({
    'simulation': {'returns_df': real_return_df,
                   'montecarlo': 10000,
                   'montecarlo_replacement': False,
                   'n_ret_years': NYEARS,
                  },
    'allocation': {},  # no args, default equal weight
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': FLOOR},
    'evaluation': {'gamma': 0},
})

print(s)

z = s.simulate(do_eval=True, return_both=False)

In [ ]:
c, bins = np.histogram([y['exhaustion'] for y in z], bins=list(range(31)))
pct_exhausted = np.sum(c[:29])/np.sum(c) * 100
print ("%.2f%% of portfolios exhausted before final year" % pct_exhausted)
bins += 1
fig = go.Figure([go.Bar(x=bins, y=c)])
fig.update_layout(showlegend=False,
                  plot_bgcolor="white",
                  title="Histogram of years to exhaustion",
                  xaxis=dict(title="Retirement Year", 
                             linecolor='black', mirror=True, ticks='inside',),
                  yaxis=dict(title="Number of Portfolios Exhausted (log scale)", 
                             linecolor='black', mirror=True, ticks='inside',
                             type="log"),
                 )

fig


In [ ]:
# Bengen 4% rule - ending val of 1st cohort = 189.255136
FIXED = 4.0
VARIABLE = 0.0
FLOOR = 4.0
NYEARS = 30

s = SWRsimulationCE({
    'simulation': {'returns_df': real_return_df,
                   'n_ret_years': NYEARS,
                  },
    'allocation': {},  # no args, default equal weight
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': FLOOR},
    'evaluation': {'gamma': 0},
})

print(s)

z = s.simulate(do_eval=True, return_both=True)
assert z[0]['trial'].iloc[0]['spend'] == 4.0, "bad value: cohort 0 year 0 spend"
assert z[0]['trial'].iloc[0]['end_port'] == pytest.approx(120.955061, 0.000001), "bad value: cohort 0 year 0 end port"
assert z[0]['trial'].iloc[-1]['spend'] == 4.0, "bad value: cohort 0 final year spend"
assert z[0]['trial'].iloc[-1]['end_port'] == pytest.approx(189.255136, 0.000001), "bad value: cohort 0 final year end port"
z[0]['trial']

In [ ]:
print('initial', np.max([s.withdrawal['fixed_pct'] + s.withdrawal['variable_pct'], s.withdrawal['floor_pct']]))
print('mean', np.mean([y['mean_spend'] for y in s.latest_simulation]))
print('worst', np.min([y['min_spend'] for y in s.latest_simulation]))
print('exhaustion pct', np.sum(np.where([y['exhaustion'] < 30 for y in s.latest_simulation], 1, 0))/30)


In [ ]:
# relaxed 4%/5% rule 
FIXED = -1
VARIABLE = 5.0
FLOOR = 4.0
NYEARS = 30

s = SWRsimulationCE({
    'simulation': {'returns_df': real_return_df,
                   'n_ret_years': NYEARS,
                  },
    'allocation': {},  # no args, default equal weight
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': FLOOR},
    'evaluation': {'gamma': 0},
})

print(s)

z = s.simulate(do_eval=True, return_both=True)
assert z[0]['trial'].iloc[0]['spend'] == pytest.approx(5.247753, 0.000001), "bad value: cohort 0 year 0 spend"
assert z[0]['trial'].iloc[0]['end_port'] == pytest.approx(119.707308, 0.000001), "bad value: cohort 0 year 0 end port"
assert z[0]['trial'].iloc[-1]['spend'] == pytest.approx(5.690874, 0.000001), "bad value: cohort 0 final year spend"
assert z[0]['trial'].iloc[-1]['end_port'] == pytest.approx(128.126609, 0.000001), "bad value: cohort 0 final year end port"
z[0]['trial']


In [ ]:
print('initial', np.max([s.withdrawal['fixed_pct'] + s.withdrawal['variable_pct'], s.withdrawal['floor_pct']]))
print('mean', np.mean([y['mean_spend'] for y in s.latest_simulation]))
print('worst', np.min([y['min_spend'] for y in s.latest_simulation]))
print('exhaustion pct', np.sum(np.where([y['exhaustion'] < 30 for y in s.latest_simulation], 1, 0))/30)


In [ ]:
# higher risk aversion rule 
FIXED = 3.5
VARIABLE = 1.1
FLOOR = 3.8
NYEARS = 30
STOCK_PCT = 0.73
BOND_PCT = 0.27

s = SWRsimulationCE({
    'simulation': {'returns_df': real_return_df,
                   'n_ret_years': NYEARS,
                  },
    'allocation': {'asset_weights': np.array([STOCK_PCT, BOND_PCT])}, 
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': FLOOR},
    'evaluation': {'gamma': 0},
})

print(s)

z = s.simulate(do_eval=True, return_both=True)
assert z[0]['trial'].iloc[0]['spend'] == pytest.approx(4.978399, 0.000001), "bad value: cohort 0 year 0 spend"
assert z[0]['trial'].iloc[0]['end_port'] == pytest.approx(129.421552, 0.000001), "bad value: cohort 0 year 0 end port"
assert z[0]['trial'].iloc[-1]['spend'] == pytest.approx(5.068669, 0.000001), "bad value: cohort 0 final year spend"
assert z[0]['trial'].iloc[-1]['end_port'] == pytest.approx(137.537621, 0.000001), "bad value: cohort 0 final year end port"
z[0]['trial']


In [ ]:
print('initial', np.max([s.withdrawal['fixed_pct'] + s.withdrawal['variable_pct'], s.withdrawal['floor_pct']]))
print('mean', np.mean([y['mean_spend'] for y in s.latest_simulation]))
print('worst', np.min([y['min_spend'] for y in s.latest_simulation]))
print('exhaustion pct', np.sum(np.where([y['exhaustion'] < 30 for y in s.latest_simulation], 1, 0))/30)


In [ ]:
# lower risk aversion rule
FIXED = 0.7
VARIABLE = 5.8
FLOOR = 3.4
NYEARS = 30
STOCK_PCT = 0.89
BOND_PCT = 0.11

s = SWRsimulationCE({
    'simulation': {'returns_df': real_return_df,
                   'n_ret_years': NYEARS,
                  },
    'allocation': {'asset_weights': np.array([STOCK_PCT, BOND_PCT])}, 
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE,
                   'floor_pct': FLOOR},
    'evaluation': {'gamma': 0},
})

print(s)

z = s.simulate(do_eval=True, return_both=True)
assert z[0]['trial'].iloc[0]['spend'] == pytest.approx(8.876278, 0.000001), "bad value: cohort 0 year 0 spend"
assert z[0]['trial'].iloc[0]['end_port'] == pytest.approx(132.094032, 0.000001), "bad value: cohort 0 year 0 end port"
assert z[0]['trial'].iloc[-1]['spend'] == pytest.approx(5.135414, 0.000001), "bad value: cohort 0 final year spend"
assert z[0]['trial'].iloc[-1]['end_port'] == pytest.approx(71.337240, 0.000001), "bad value: cohort 0 final year end port"
z[0]['trial']



In [ ]:
print('initial', np.max([s.withdrawal['fixed_pct'] + s.withdrawal['variable_pct'], s.withdrawal['floor_pct']]))
print('mean', np.mean([y['mean_spend'] for y in s.latest_simulation]))
print('worst', np.min([y['min_spend'] for y in s.latest_simulation]))
print('exhaustion pct', np.sum(np.where([y['exhaustion'] < 30 for y in s.latest_simulation], 1, 0))/30)
